# Collect Census Data in Python

Collect Census data with [Census API](https://www.census.gov/data/developers/data-sets.html) and store the information to a PostgreSQL database. 

## Install Python Packages  

- census: a Census API wrapper to call Census APIs
- us: a package praovidng US states antd territories names and FIPS codes
- psycopg2: PostgreSQL databases adapter to create tables, insert data, and execute queries

In [1]:
pip install census

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install us

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install psycopg2

Note: you may need to restart the kernel to use updated packages.


## Secrete Manager Function

In [4]:
import boto3
from botocore.exceptions import ClientError
import json

def get_secret(secret_name):
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e

    secret = get_secret_value_response['SecretString']
    
    return json.loads(secret)

## Connect to Database

In [12]:
import psycopg2

conn = psycopg2.connect(host=get_secret('postgresql')['host'],
                      user=get_secret('postgresql')['username'],
                      password = get_secret('postgresql')['password'],
                      dbname=get_secret('postgresql')['engine'])

cur = conn.cursor()

## Create Tables

In [18]:
sql = """
-- Create tables
CREATE TABLE IF NOT EXISTS name
(
    fips VARCHAR(4) NOT NULL UNIQUE,
    name VARCHAR(100) UNIQUE,
    PRIMARY KEY(fips)
);

CREATE TABLE IF NOT EXISTS population
(
    fips VARCHAR(4) NOT NULL,
    pop INTEGER NOT NULL,
    year INTEGER NOT NULL,
    PRIMARY KEY(fips, year)
);

CREATE TABLE IF NOT EXISTS income
(
    fips VARCHAR(4) NOT NULL,
    income INTEGER NOT NULL,
    year INTEGER NOT NULL,
    PRIMARY KEY(fips, year)
);


-- Create FKs
ALTER TABLE population
    ADD    FOREIGN KEY (fips)
    REFERENCES name(fips)
    MATCH SIMPLE
;
    
ALTER TABLE income
    ADD    FOREIGN KEY (fips)
    REFERENCES name(fips)
    MATCH SIMPLE
;

"""

In [19]:
cur.execute(sql)
conn.commit()

InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block


## Insert Census Data into Tables

Load Census API keys

In [26]:
from census import Census
from us import states


c = Census('405b36535ed0b119cd396144825a73e9e97cd9d0')

Insert state names to the name table.

In [28]:
for state in states.STATES_AND_TERRITORIES:
    sql = f"""insert into name (name,fips) 
            values ('{state}','{state.fips}') """
    cur.execute(sql)
    
sql = f"""insert into name (name,fips) 
            values ('{states.DC.name}','{states.DC.fips}') """
cur.execute(sql)

conn.commit()

InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block


Insert the population data to the population table.

In [25]:
for year in range(2005,2020):
    print('collecting data in year of: ',year)
    for data in c.acs1.get('B01003_001E',{'for': 'state:*'},year=year):
        
        sql = f"""insert into population (fips,pop,year) 
            values ('{data['state']}',{data['B01003_001E']},{year}) """
#         print(sql)
        cur.execute(sql)
    conn.commit()

collecting data in year of:  2005


APIKeyError: ' <html style="font-size: 14px;">     <head>         <title>Invalid Key</title>         <link rel="icon" type="image/x-icon" href="favicon.ico">         <link rel="stylesheet" type="text/css" href="assets/styles.css">         <script type="text/javascript" src="assets/jquery-1.4.4.min.js"></script>         <script type="text/javascript">             $(document).ready(function() {                 $(".menu-activator").click(function() {                     $(".gov-menu").toggle()                     $(".chevron").toggleClass(\'rotate\')                 })                             })         </script>     </head>     <body>         <header>             <div data-v-8adc6fed="" data-v-2f72e816="" class="gov-banner noprint">                 <div data-v-8adc6fed="" class="aqua-layout horizontal align-start justify-start banner-header">                     <img data-v-8adc6fed="" src="data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAABAAAAALCAMAAABBPP0LAAAAG1BMVEUdM7EeNLIeM7HgQCDa

Insert the median household income into the income table.

In [29]:
for year in range(2005,2020):
    print('collecting data in year of: ',year)
    for data in c.acs1.get('B19013_001E',{'for': 'state:*'},year=year):
        
        sql = f"""insert into income (fips,income,year) 
            values ('{data['state']}',{data['B19013_001E']},{year}) """
#         print(sql)
        cur.execute(sql)
    conn.commit()


collecting data in year of:  2005


APIKeyError: ' <html style="font-size: 14px;">     <head>         <title>Invalid Key</title>         <link rel="icon" type="image/x-icon" href="favicon.ico">         <link rel="stylesheet" type="text/css" href="assets/styles.css">         <script type="text/javascript" src="assets/jquery-1.4.4.min.js"></script>         <script type="text/javascript">             $(document).ready(function() {                 $(".menu-activator").click(function() {                     $(".gov-menu").toggle()                     $(".chevron").toggleClass(\'rotate\')                 })                             })         </script>     </head>     <body>         <header>             <div data-v-8adc6fed="" data-v-2f72e816="" class="gov-banner noprint">                 <div data-v-8adc6fed="" class="aqua-layout horizontal align-start justify-start banner-header">                     <img data-v-8adc6fed="" src="data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAABAAAAALCAMAAABBPP0LAAAAG1BMVEUdM7EeNLIeM7HgQCDa

## Close Dabase Connection

In [9]:
cur.close()
conn.close()